---
# AIET Capstone Project: Group 2

## Northeastern SMILE Lab - Recognizing Faces in the Wild

https://www.kaggle.com/c/recognizing-faces-in-the-wild/kernels

---

### Data Cleaning and Preprocessing:

---

In [595]:
# dependencies
import os
import numpy as np
import pandas as pd
import glob
import random
import base64

from PIL import Image
from io import BytesIO
from IPython.display import HTML

In [596]:
root_path = os.getcwd()
root_path

'/Users/harish3110/Desktop/capstone_project'

#### Dataset Summary:

In [597]:
fam = os.listdir(root_path + "/train")
ind = []
num = []
pic = []
tot = 0
totpic = 0
for i in fam:
    try:
        path = root_path + "/train/"+ str(i)
        temp = os.listdir(path)
        ind.append(temp)
        num.append(len(temp))
        tot+=len(temp)
        for j in temp:
            newpath = path+"/"+str(j)
            temp = os.listdir(newpath)
            pic.append(temp)
            totpic+=len(temp)
    except:
        if str(i) == ".DS_Store":
            continue
print(f'We have {len(fam)} families, with {tot} distinct people having a total of {totpic} pictures.') 

We have 471 families, with 2320 distinct people having a total of 12348 pictures.


---

In [598]:
from collections import defaultdict
#keeps all photos path in a dictionary

allPhotos = defaultdict(list)
for family in glob.glob(root_path + "/train/*"):
    for mem in glob.glob(family+'/*'):
        for photo in glob.glob(mem+'/*'):
            allPhotos[mem].append(photo)

#list of all members with valid photo
ppl = list(allPhotos.keys())
len(ppl)

2316

In [599]:
data = pd.read_csv(root_path + '/train_relationships.csv')
data.p1 = data.p1.apply( lambda x: root_path + '/train/'+x )
data.p2 = data.p2.apply( lambda x: root_path + '/train/'+x )
print(data.shape)
data.head()

(3598, 2)


,p1,p2
0,/Users/harish3110/Desktop/capstone_project/tra...,/Users/harish3110/Desktop/capstone_project/tra...
1,/Users/harish3110/Desktop/capstone_project/tra...,/Users/harish3110/Desktop/capstone_project/tra...
2,/Users/harish3110/Desktop/capstone_project/tra...,/Users/harish3110/Desktop/capstone_project/tra...
3,/Users/harish3110/Desktop/capstone_project/tra...,/Users/harish3110/Desktop/capstone_project/tra...
4,/Users/harish3110/Desktop/capstone_project/tra...,/Users/harish3110/Desktop/capstone_project/tra...


---
#### Dropping duplicates between data and combinations(i.e all possible combinations between all memebers in a particular family):
##### Didn't work as expected!

In [600]:
from itertools import combinations
l2 = []
for family in glob.glob(root_path + "/train/*"):
    l = []
    for mem in glob.glob(family+'/*'):
        l.append(mem)
    l2.append(list(combinations(l, 2)))
        
l2[1]

[('/Users/harish3110/Desktop/capstone_project/train/F0658/MID6',
  '/Users/harish3110/Desktop/capstone_project/train/F0658/MID1'),
 ('/Users/harish3110/Desktop/capstone_project/train/F0658/MID6',
  '/Users/harish3110/Desktop/capstone_project/train/F0658/MID7'),
 ('/Users/harish3110/Desktop/capstone_project/train/F0658/MID6',
  '/Users/harish3110/Desktop/capstone_project/train/F0658/MID2'),
 ('/Users/harish3110/Desktop/capstone_project/train/F0658/MID6',
  '/Users/harish3110/Desktop/capstone_project/train/F0658/MID5'),
 ('/Users/harish3110/Desktop/capstone_project/train/F0658/MID6',
  '/Users/harish3110/Desktop/capstone_project/train/F0658/MID3'),
 ('/Users/harish3110/Desktop/capstone_project/train/F0658/MID1',
  '/Users/harish3110/Desktop/capstone_project/train/F0658/MID7'),
 ('/Users/harish3110/Desktop/capstone_project/train/F0658/MID1',
  '/Users/harish3110/Desktop/capstone_project/train/F0658/MID2'),
 ('/Users/harish3110/Desktop/capstone_project/train/F0658/MID1',
  '/Users/harish31

In [601]:
combinations = [l2[i][j] for i in range(len(l2)) for j in range(len(l2[i]))]
combinations[0][0]

'/Users/harish3110/Desktop/capstone_project/train/F0832/MID1'

In [602]:
combinations = pd.DataFrame(combinations, columns=['p1', 'p2'])
len(combinations)

6260

In [603]:
len(data)

3598

In [604]:
df1 = combinations
df2 = data

In [605]:
# Dropping duplicates between data and combinations. Didn't work as expected!
pd.concat([df1, df2]).drop_duplicates(keep=False).head()

,p1,p2
0,/Users/harish3110/Desktop/capstone_project/tra...,/Users/harish3110/Desktop/capstone_project/tra...
5,/Users/harish3110/Desktop/capstone_project/tra...,/Users/harish3110/Desktop/capstone_project/tra...
7,/Users/harish3110/Desktop/capstone_project/tra...,/Users/harish3110/Desktop/capstone_project/tra...
8,/Users/harish3110/Desktop/capstone_project/tra...,/Users/harish3110/Desktop/capstone_project/tra...
9,/Users/harish3110/Desktop/capstone_project/tra...,/Users/harish3110/Desktop/capstone_project/tra...


In [606]:
nr1=pd.concat([df2, df1, df1]).drop_duplicates(keep=False)
nr2=pd.concat([df1, df2, df2]).drop_duplicates(keep=False)

In [607]:
len(nr1)

1664

In [608]:
print(nr1.iloc[0][0])
print(nr1.iloc[0][1])

/Users/harish3110/Desktop/capstone_project/train/F0005/MID3
/Users/harish3110/Desktop/capstone_project/train/F0005/MID2


In [609]:
nr1['p1t'] = nr1.p1
nr1.head()

,p1,p2,p1t
3,/Users/harish3110/Desktop/capstone_project/tra...,/Users/harish3110/Desktop/capstone_project/tra...,/Users/harish3110/Desktop/capstone_project/tra...
7,/Users/harish3110/Desktop/capstone_project/tra...,/Users/harish3110/Desktop/capstone_project/tra...,/Users/harish3110/Desktop/capstone_project/tra...
9,/Users/harish3110/Desktop/capstone_project/tra...,/Users/harish3110/Desktop/capstone_project/tra...,/Users/harish3110/Desktop/capstone_project/tra...
11,/Users/harish3110/Desktop/capstone_project/tra...,/Users/harish3110/Desktop/capstone_project/tra...,/Users/harish3110/Desktop/capstone_project/tra...
12,/Users/harish3110/Desktop/capstone_project/tra...,/Users/harish3110/Desktop/capstone_project/tra...,/Users/harish3110/Desktop/capstone_project/tra...


In [610]:
nr1.p1 = nr1.p2
nr1.p2 = nr1.p1t

nr1.drop(['p1t'], axis=1, inplace=True)
nr1.head()

,p1,p2
3,/Users/harish3110/Desktop/capstone_project/tra...,/Users/harish3110/Desktop/capstone_project/tra...
7,/Users/harish3110/Desktop/capstone_project/tra...,/Users/harish3110/Desktop/capstone_project/tra...
9,/Users/harish3110/Desktop/capstone_project/tra...,/Users/harish3110/Desktop/capstone_project/tra...
11,/Users/harish3110/Desktop/capstone_project/tra...,/Users/harish3110/Desktop/capstone_project/tra...
12,/Users/harish3110/Desktop/capstone_project/tra...,/Users/harish3110/Desktop/capstone_project/tra...


In [611]:
print(nr1.iloc[0][0])
print(nr1.iloc[0][1])

/Users/harish3110/Desktop/capstone_project/train/F0005/MID2
/Users/harish3110/Desktop/capstone_project/train/F0005/MID3


In [612]:
len(nr2)

4326

In [613]:
len(pd.concat([nr2, nr1, nr1]).drop_duplicates(keep=False))

2893

---

In [614]:
import matplotlib.image as mpimg

In [615]:
# TEST & TRAIN FILES
train_df = pd.read_csv(root_path + '/train_relationships.csv')
test_df = pd.read_csv(root_path + '/sample_submission.csv')

# BASE PATHS FOR IMAGES
train_img_path = root_path + '/train/'
test_img_path = root_path + '/test/'

In [616]:
train_df = data

In [617]:
# SETUP SOME COLUMNS TO USE LATER
train_df['is_related'] = 1
train_df['p1_photo_count'] = 0
train_df['p2_photo_count'] = 0
train_df['p1_photo_list'] = ''
train_df['p2_photo_list'] = ''
train_df['combo'] = ''

print(train_df.shape[0], ' rows of confirmed kinship for model training')

# LOOP THROUGH EACH KNOWN KINSHIP
for i in range(0, train_df.shape[0]) :
    p1_path = root_path + '/train/' + train_df.loc[i, 'p1'] + "/"
    p2_path = root_path + '/train/' + train_df.loc[i, 'p2'] + "/"

    # CHECK IF DIRECTORY EXISTS (sometimes it's not present, unless I've done something wrong)
    if os.path.isdir(p1_path) & os.path.isdir(p2_path):
        train_df.loc[i, 'p1_photo_count'] = len(os.listdir(train_img_path / train_df.loc[i, 'p1'] ))
        train_df.loc[i, 'p2_photo_count'] = len(os.listdir(train_img_path / train_df.loc[i, 'p2'] ))
        train_df.at[i, 'p1_photo_list'] = os.listdir(train_img_path / train_df.loc[i, 'p1'] )
        train_df.at[i, 'p2_photo_list'] = os.listdir(train_img_path / train_df.loc[i, 'p2'] )
        train_df.loc[i, 'combo'] = train_df.loc[i, 'p1'] + '-' + train_df.loc[i, 'p2']

train_columns = list(train_df.columns.values)

3598  rows of confirmed kinship for model training


In [618]:
train_df.head()

,p1,p2,is_related,p1_photo_count,p2_photo_count,p1_photo_list,p2_photo_list,combo
0,/Users/harish3110/Desktop/capstone_project/tra...,/Users/harish3110/Desktop/capstone_project/tra...,1,0,0,,,
1,/Users/harish3110/Desktop/capstone_project/tra...,/Users/harish3110/Desktop/capstone_project/tra...,1,0,0,,,
2,/Users/harish3110/Desktop/capstone_project/tra...,/Users/harish3110/Desktop/capstone_project/tra...,1,0,0,,,
3,/Users/harish3110/Desktop/capstone_project/tra...,/Users/harish3110/Desktop/capstone_project/tra...,1,0,0,,,
4,/Users/harish3110/Desktop/capstone_project/tra...,/Users/harish3110/Desktop/capstone_project/tra...,1,0,0,,,


In [619]:
from tqdm import tqdm

# GET A LIST OF ALL INDIVIDUALS IN THE TRAINING DATA
train_individuals = list(set(list(train_df['p1']) + list(train_df['p2'])))
print(len(train_individuals), ' individuals in training dataset')
print(len(individual_list), ' individuals from the 3rd party data')
all_individuals = train_individuals + individual_list
print(len(all_individuals), ' total individuals in the new training data')

# GENERATE ALL POSSIBLE COMBINATIONS OF INDIVIDUALS
combinations = list(itertools.combinations(train_individuals + individual_list, 2))

print(len(combinations), ' possible 2-way combinations of individuals')

# CLEAR OUT ANY RELATED COMBINATIONS
for i in tqdm(range(0, train_df.shape[0])):
    combo = ( train_df.loc[i, 'p1'], train_df.loc[i, 'p2'] )
    if combo in combinations:
        combinations.remove(combo)
print(len(combinations), ' possible 2-way combinations of individuals after removing duplicates/related people')

2412  individuals in training dataset
0  individuals from the 3rd party data
2412  total individuals in the new training data
2907666  possible 2-way combinations of individuals


100%|██████████| 3598/3598 [05:51<00:00,  6.98it/s]

2905909  possible 2-way combinations of individuals after removing duplicates/related people


In [620]:
type(combinations)

list

In [621]:
combinations[:10]

[('/Users/harish3110/Desktop/capstone_project/train/F0770/MID1',
  '/Users/harish3110/Desktop/capstone_project/train/F0411/MID2'),
 ('/Users/harish3110/Desktop/capstone_project/train/F0770/MID1',
  '/Users/harish3110/Desktop/capstone_project/train/F0591/MID3'),
 ('/Users/harish3110/Desktop/capstone_project/train/F0770/MID1',
  '/Users/harish3110/Desktop/capstone_project/train/F0313/MID2'),
 ('/Users/harish3110/Desktop/capstone_project/train/F0770/MID1',
  '/Users/harish3110/Desktop/capstone_project/train/F0601/MID38'),
 ('/Users/harish3110/Desktop/capstone_project/train/F0770/MID1',
  '/Users/harish3110/Desktop/capstone_project/train/F0231/MID14'),
 ('/Users/harish3110/Desktop/capstone_project/train/F0770/MID1',
  '/Users/harish3110/Desktop/capstone_project/train/F0051/MID6'),
 ('/Users/harish3110/Desktop/capstone_project/train/F0770/MID1',
  '/Users/harish3110/Desktop/capstone_project/train/F0690/MID1'),
 ('/Users/harish3110/Desktop/capstone_project/train/F0770/MID1',
  '/Users/harish

In [622]:
import pickle
with open("combinations.txt", "wb") as fp:   #Pickling
    pickle.dump(combinations, fp)

In [623]:
with open("combinations.txt", "rb") as fp:   # Unpickling
    combinations1 = pickle.load(fp)

In [624]:
combinations1[:10]

[('/Users/harish3110/Desktop/capstone_project/train/F0770/MID1',
  '/Users/harish3110/Desktop/capstone_project/train/F0411/MID2'),
 ('/Users/harish3110/Desktop/capstone_project/train/F0770/MID1',
  '/Users/harish3110/Desktop/capstone_project/train/F0591/MID3'),
 ('/Users/harish3110/Desktop/capstone_project/train/F0770/MID1',
  '/Users/harish3110/Desktop/capstone_project/train/F0313/MID2'),
 ('/Users/harish3110/Desktop/capstone_project/train/F0770/MID1',
  '/Users/harish3110/Desktop/capstone_project/train/F0601/MID38'),
 ('/Users/harish3110/Desktop/capstone_project/train/F0770/MID1',
  '/Users/harish3110/Desktop/capstone_project/train/F0231/MID14'),
 ('/Users/harish3110/Desktop/capstone_project/train/F0770/MID1',
  '/Users/harish3110/Desktop/capstone_project/train/F0051/MID6'),
 ('/Users/harish3110/Desktop/capstone_project/train/F0770/MID1',
  '/Users/harish3110/Desktop/capstone_project/train/F0690/MID1'),
 ('/Users/harish3110/Desktop/capstone_project/train/F0770/MID1',
  '/Users/harish

In [625]:
def get_profile(individual):
    # ORIGINAL DATASET
    print (len(train_df[train_df['p1'] == individual]['p1_photo_count'].head(1).values))
    if len(train_df[train_df['p1'] == individual]['p1_photo_count'].head(1).values) == 0:
        return {
            'photo_list': train_df[train_df['p2'] == individual]['p2_photo_list'].head(1).values[0],
            'photo_count': train_df[train_df['p2'] == individual]['p2_photo_count'].head(1).values[0]
        }
    else:
        return {                                                        # it's not stupid if it works
            'photo_list': train_df[train_df['p1'] == individual]['p1_photo_list'].head(1).values[0],
            'photo_count': train_df[train_df['p1'] == individual]['p1_photo_count'].head(1).values[0]
        }

In [626]:
goal_n = 10000  # 10k this gives us a 64/36-ish ratio for our target of 'is_related'

print(goal_n - train_df.shape[0], ' new rows to be created for training dataset')

# generating a list of random numbers for indexes to grab from the combinations list
random_list = []
for i in range(0, goal_n - train_df.shape[0]): # for each of the number of rows I need to add
    x = random.randint(0, len(combinations))   # generate a random number between 0 and combinations length
    random_list.append(x)

rows_list = []

for i in range(0, goal_n - train_df.shape[0]):

    combo = combinations[random_list[i]][0] + '-' + combinations[random_list[i]][1]
    
    p1 = get_profile(combo.split('-')[0])
    p2 = get_profile(combo.split('-')[1])

    new_row = {
        'p1': combinations[random_list[i]][0],
        'p2': combinations[random_list[i]][1],
        'is_related': 0, 
        'p1_photo_count': p1['photo_count'], 
        'p2_photo_count': p2['photo_count'], 
        'p1_photo_list': p1['photo_list'], 
        'p2_photo_list': p2['photo_list'],  
        'combo': combo
    }
    
    rows_list.append(new_row)

new_rows = pd.DataFrame(rows_list)

6402  new rows to be created for training dataset
1
0
1
1
1
1
1
1
1
0
1
0
0
1
1
0
1
0
1
0
0
1
1
0
1
0
1
1
0
1
0
1
0
0
0
0
0
1
0
0
0
1
0
1
1
1
1
1
0
1
0
1
1
0
1
1
1
1
0
0
0
1
1
1
1
0
0
1
1
1
1
1
1
1
0
1
1
0
0
0
0
0
1
1
1
1
1
0
1
1
0
1
1
1
1
1
1
0
1
0
1
0
0
1
0
1
1
1
1
0
1
1
1
1
0
1
1
1
1
0
1
1
1
0
1
0
1
1
1
1
0
1
0
0
0
1
0
1
0
1
1
1
0
1
1
0
0
1
1
1
1
0
0
0
1
1
1
1
0
1
1
1
1
1
1
0
0
0
0
0
1
1
1
1
1
1
0
1
0
0
1
0
0
1
1
1
1
0
0
0
0
1
1
1
1
1
0
1
1
0
0
0
1
1
1
1
0
1
1
0
1
1
1
1
1
1
0
1
1
0
1
1
1
1
0
0
0
1
1
1
1
1
0
1
1
1
0
1
1
0
1
1
1
0
1
1
1
0
1
1
1
1
1
1
0
1
1
1
0
1
0
1
1
1
1
1
0
1
0
0
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
0
0
1
1
1
1
0
1
1
1
0
1
1
1
1
1
0
0
1
0
1
1
0
1
1
1
0
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
0
1
1
0
1
1
1
0
0
1
1
1
0
0
0
0
0
1
0
0
1
1
1
1
0
1
0
1
0
1
1
1
0
0
1
1
0
0
1
1
1
1
0
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
0
1
0
0
1
0
1
0
1
1
0
1
1
0
1
1
1
1
0
1
1
0
0
1
1
0
1
1
0
1
0
1
1
1
0
1
0
1
1
0
1
1
0
0
0
1
0
0
0
1
1
0
0
1
1
0
1
0
1
1
1
0
1
1
1
1
1
1
1
1
1


In [627]:
if train_df.shape[0] <= goal_n:
    train_df = pd.concat( [ train_df, new_rows ], ignore_index=True )


/Library/Frameworks/Python.framework/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [628]:
train_df.head()

,combo,is_related,p1,p1_photo_count,p1_photo_list,p2,p2_photo_count,p2_photo_list
0,,1,/Users/harish3110/Desktop/capstone_project/tra...,0,,/Users/harish3110/Desktop/capstone_project/tra...,0,
1,,1,/Users/harish3110/Desktop/capstone_project/tra...,0,,/Users/harish3110/Desktop/capstone_project/tra...,0,
2,,1,/Users/harish3110/Desktop/capstone_project/tra...,0,,/Users/harish3110/Desktop/capstone_project/tra...,0,
3,,1,/Users/harish3110/Desktop/capstone_project/tra...,0,,/Users/harish3110/Desktop/capstone_project/tra...,0,
4,,1,/Users/harish3110/Desktop/capstone_project/tra...,0,,/Users/harish3110/Desktop/capstone_project/tra...,0,


In [629]:
train_df.is_related[train_df.is_related == 0].count()

6402

In [630]:
train_df.is_related[train_df.is_related == 1].count()

3598

In [631]:
train_df.head()

,combo,is_related,p1,p1_photo_count,p1_photo_list,p2,p2_photo_count,p2_photo_list
0,,1,/Users/harish3110/Desktop/capstone_project/tra...,0,,/Users/harish3110/Desktop/capstone_project/tra...,0,
1,,1,/Users/harish3110/Desktop/capstone_project/tra...,0,,/Users/harish3110/Desktop/capstone_project/tra...,0,
2,,1,/Users/harish3110/Desktop/capstone_project/tra...,0,,/Users/harish3110/Desktop/capstone_project/tra...,0,
3,,1,/Users/harish3110/Desktop/capstone_project/tra...,0,,/Users/harish3110/Desktop/capstone_project/tra...,0,
4,,1,/Users/harish3110/Desktop/capstone_project/tra...,0,,/Users/harish3110/Desktop/capstone_project/tra...,0,


In [632]:
train_df.drop(['combo', 'p1_photo_count', 'p1_photo_list', 'p2_photo_count', 'p2_photo_list'], axis=1, inplace=True)

train_df.head()

,is_related,p1,p2
0,1,/Users/harish3110/Desktop/capstone_project/tra...,/Users/harish3110/Desktop/capstone_project/tra...
1,1,/Users/harish3110/Desktop/capstone_project/tra...,/Users/harish3110/Desktop/capstone_project/tra...
2,1,/Users/harish3110/Desktop/capstone_project/tra...,/Users/harish3110/Desktop/capstone_project/tra...
3,1,/Users/harish3110/Desktop/capstone_project/tra...,/Users/harish3110/Desktop/capstone_project/tra...
4,1,/Users/harish3110/Desktop/capstone_project/tra...,/Users/harish3110/Desktop/capstone_project/tra...


In [634]:
train_df = train_df.sample(frac=1).reset_index(drop=True)
train_df.head()

,is_related,p1,p2
0,0,/Users/harish3110/Desktop/capstone_project/tra...,/Users/harish3110/Desktop/capstone_project/tra...
1,1,/Users/harish3110/Desktop/capstone_project/tra...,/Users/harish3110/Desktop/capstone_project/tra...
2,1,/Users/harish3110/Desktop/capstone_project/tra...,/Users/harish3110/Desktop/capstone_project/tra...
3,0,/Users/harish3110/Desktop/capstone_project/tra...,/Users/harish3110/Desktop/capstone_project/tra...
4,0,/Users/harish3110/Desktop/capstone_project/tra...,/Users/harish3110/Desktop/capstone_project/tra...


In [635]:
train_df.to_csv('df_final.csv')